In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Coronary_artery_disease"
cohort = "GSE64566"

# Input paths
in_trait_dir = "../../input/GEO/Coronary_artery_disease"
in_cohort_dir = "../../input/GEO/Coronary_artery_disease/GSE64566"

# Output paths
out_data_file = "../../output/preprocess/Coronary_artery_disease/GSE64566.csv"
out_gene_data_file = "../../output/preprocess/Coronary_artery_disease/gene_data/GSE64566.csv"
out_clinical_data_file = "../../output/preprocess/Coronary_artery_disease/clinical_data/GSE64566.csv"
json_path = "../../output/preprocess/Coronary_artery_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"GE/miRNA expression profile of Human Epicardial Adipose Tissue (EAT) and Subcutaneous Adipose Tissue (SAT) in Patients with Coronary Artery Disease (CAD) vs. Controls (CTRL)"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['age: 76', 'age: 71', 'age: 57', 'age: 30', 'age: 81', 'age: 51', 'age: 36', 'age: 44', 'age: 52', 'age: 73', 'age: 69', 'age: 56', 'age: 70', 'age: 67', 'age: 60', 'age: 61', 'age: 86', 'age: 82'], 1: ['tissue: Subcutaneous Adipose', 'tissue: Epicardial Adipose'], 2: ['disease state: control', 'disease state: coronary artery disease']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains expression profiles.
# However, it seems to be a SuperSeries that may contain both GE (gene expression) and miRNA data.
# Since we don't have specific information about gene expression data, we'll treat it as potentially available.
is_gene_available = True

# 2. Analyze available clinical features
# 2.1 Identify rows for trait, age, and gender
# For trait (coronary artery disease):
trait_row = 0  # 'disease state' contains CAD vs control information
# For age:
age_row = 1  # 'age' information is available
# For gender:
gender_row = None  # No gender information found in the sample characteristics

# 2.2 Define conversion functions
def convert_trait(value):
    """Convert disease state to binary (0 for control, 1 for CAD)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if 'disease state:' in value:
        actual_value = value.split(':')[1].strip()
        if 'control' in actual_value:
            return 0
        elif 'coronary artery disease' in actual_value:
            return 1
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    if 'age:' in value:
        try:
            age = int(value.split(':')[1].strip())
            return age
        except:
            return None
    return None

# No gender conversion function needed as gender data is not available
convert_gender = None

# 3. Save metadata about data availability
is_trait_available = trait_row is not None
validation_result = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM1574149': [nan, nan], 'GSM1574150': [nan, nan], 'GSM1574151': [nan, nan], 'GSM1574152': [nan, nan], 'GSM1574153': [nan, nan], 'GSM1574154': [nan, nan], 'GSM1574155': [nan, nan], 'GSM1574156': [nan, nan], 'GSM1574157': [nan, nan], 'GSM1574158': [nan, nan], 'GSM1574159': [nan, nan], 'GSM1574160': [nan, nan], 'GSM1574161': [nan, nan], 'GSM1574162': [nan, nan], 'GSM1574163': [nan, nan], 'GSM1574164': [nan, nan], 'GSM1574165': [nan, nan], 'GSM1574166': [nan, nan], 'GSM1574167': [nan, nan], 'GSM1574168': [nan, nan], 'GSM1574169': [nan, nan], 'GSM1574170': [nan, nan], 'GSM1574171': [nan, nan], 'GSM1574172': [nan, nan], 'GSM1574173': [nan, nan], 'GSM1574174': [nan, nan], 'GSM1574175': [nan, nan], 'GSM1574176': [nan, nan], 'GSM1574177': [nan, nan], 'GSM1574178': [nan, nan], 'GSM1574179': [nan, nan], 'GSM1574180': [nan, nan], 'GSM1574181': [nan, nan], 'GSM1574182': [nan, nan], 'GSM1574183': [nan, nan], 'GSM1574184': [nan, nan], 'GSM1574185': [nan, nan], 'GSM1574186

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Coronary_artery_disease/GSE64566/GSE64566_family.soft.gz
Matrix file: ../../input/GEO/Coronary_artery_disease/GSE64566/GSE64566-GPL6947_series_matrix.txt.gz
Found the matrix table marker at line 56


Gene data shape: (48783, 46)


First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers are from Illumina BeadArray platform (ILMN_ prefix)
# These are probe IDs that need to be mapped to human gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, the Symbol column seems to contain gene information
print("\nExamining ID and Symbol columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'Symbol' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}")
        print(f"Symbol: {gene_annotation['Symbol'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['Symbol'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nSymbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")
    
    # Check if some extracted gene symbols can be found in the Symbol column
    print("\nAttempting to extract gene symbols from the first few rows:")
    for i in range(min(3, len(gene_annotation))):
        if pd.notna(gene_annotation['Symbol'].iloc[i]):
            symbols = extract_human_gene_symbols(str(gene_annotation['Symbol'].iloc[i]))
            print(f"Row {i} extracted symbols: {symbols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'nuID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1'], 'RefSeq_ID': ['XM_933824.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for gene mapping
# Based on the preview, we observed:
# - Gene expression data has identifiers like 'ILMN_3166935'
# - Gene annotation data has 'ID' column with identifiers like 'ILMN_1725881'
# - Gene annotation data has 'Symbol' column with gene symbols like 'FCGR2B'

# Check if we have the required columns for mapping
if 'ID' in gene_annotation.columns and 'Symbol' in gene_annotation.columns:
    print("Creating gene mapping from 'ID' to 'Symbol'")
    
    # 2. Get a gene mapping dataframe
    # Create a custom mapping dataframe that preserves more gene symbols
    mapping_df = gene_annotation.loc[:, ['ID', 'Symbol']].dropna()
    # Some cleaning to ensure ID is string type
    mapping_df = mapping_df.astype({'ID': 'str'})
    # Ensure correct column names for the apply_gene_mapping function
    mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})
    
    print(f"Gene mapping dataframe shape: {mapping_df.shape}")
    print(f"First few rows of mapping_df:\n{mapping_df.head()}")
    
    # Check for overlap between gene expression data and annotation data
    expression_ids = set(gene_data.index)
    mapping_ids = set(mapping_df['ID'])
    overlap_ids = expression_ids.intersection(mapping_ids)
    
    print(f"Number of probe IDs in expression data: {len(expression_ids)}")
    print(f"Number of probe IDs in mapping data: {len(mapping_ids)}")
    print(f"Number of overlapping probe IDs: {len(overlap_ids)}")
    
    if len(overlap_ids) == 0:
        print("ERROR: No overlap between expression data probe IDs and annotation probe IDs.")
        print("This indicates a platform mismatch - the annotation file doesn't match the expression data.")
        is_gene_available = False
    else:
        # Create a modified version of apply_gene_mapping that doesn't filter gene symbols
        def modified_apply_gene_mapping(expression_df, mapping_df):
            """Custom version to retain more gene symbols"""
            # Keep only the mappings that exist in the expression data
            mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
            
            # Count genes per probe
            mapping_df['num_genes'] = 1  # Each probe maps to one gene
            mapping_df.set_index('ID', inplace=True)
            
            # Merge and distribute expression values
            merged_df = mapping_df.join(expression_df)
            expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
            merged_df[expr_cols] = merged_df[expr_cols].div(merged_df['num_genes'], axis=0)
            
            # Sum expression values for each gene
            gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()
            return gene_expression_df
        
        # Apply the modified gene mapping approach
        gene_data = modified_apply_gene_mapping(gene_data, mapping_df)
        print(f"Gene expression data shape after mapping: {gene_data.shape}")
        
        if gene_data.shape[0] == 0:
            print("ERROR: Gene mapping resulted in empty dataframe.")
            is_gene_available = False
        else:
            print(f"First few gene symbols after mapping:\n{gene_data.index[:5].tolist()}")
            
            # Normalize gene symbols
            gene_data = normalize_gene_symbols_in_index(gene_data)
            print(f"Gene expression data shape after normalization: {gene_data.shape}")
            print(f"First few normalized gene symbols:\n{gene_data.index[:5].tolist()}")
            
            # Save the gene data
            os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
            gene_data.to_csv(out_gene_data_file)
            print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("Required columns for gene mapping not found in annotation data.")
    is_gene_available = False

# Update validation if gene mapping failed
if not is_gene_available:
    print("Gene data mapping failed - cannot proceed with further analysis.")
    is_trait_available = True  # From previous steps
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Creating gene mapping from 'ID' to 'Symbol'
Gene mapping dataframe shape: (36892, 2)
First few rows of mapping_df:
             ID       Gene
0  ILMN_1725881   LOC23117
2  ILMN_1804174     FCGR2B
3  ILMN_1796063     TRIM44
4  ILMN_1811966  LOC653895
5  ILMN_1668162    DGAT2L3
Number of probe IDs in expression data: 48783
Number of probe IDs in mapping data: 36892
Number of overlapping probe IDs: 36138
Gene expression data shape after mapping: (25150, 46)
First few gene symbols after mapping:
['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1']


Gene expression data shape after normalization: (18550, 46)
First few normalized gene symbols:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2']


Gene expression data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE64566.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Attempt to load gene data and handle possible issues with normalization
try:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Check if gene_data (from previous step) has any content
    if gene_data.shape[0] == 0:
        print("WARNING: Gene data is empty after normalization in previous step.")
        print("This appears to be miRNA data rather than gene expression data.")
        
        # Since gene_data is empty, set gene_available to False
        is_gene_available = False
        
        # Create an empty dataframe for metadata purposes
        empty_df = pd.DataFrame()
        
        # Log information about this dataset for future reference
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Consider it biased as we can't use it
            df=empty_df,
            note="Dataset appears to contain miRNA data rather than gene expression data. Gene symbols could not be normalized."
        )
        
        print("Dataset marked as unusable due to lack of valid gene expression data.")
    else:
        # If gene_data is not empty, proceed with normalization and linking
        print(f"Gene data shape before normalization: {gene_data.shape}")
        
        # Save the gene data we have, even if it's already normalized
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
        
        # Attempt to link clinical and gene data
        if is_trait_available:
            # Load clinical data
            clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
            print(f"Loaded clinical data shape: {clinical_features.shape}")
            
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate data quality and save cohort info
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note="Successfully processed gene expression data for coronary artery disease."
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            # Cannot proceed with linking if trait data is missing
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="Cannot link data because trait information is not available."
            )
except Exception as e:
    print(f"Error in data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # Consider gene data unavailable if we had an error
        is_trait_available=is_trait_available,
        is_biased=True,  # Consider it biased as we can't use it
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note=f"Error during normalization or linking: {str(e)}"
    )

Gene data shape before normalization: (18550, 46)


Gene data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE64566.csv
Loaded clinical data shape: (2, 46)
Initial linked data shape: (46, 18552)
Linked data shape after handling missing values: (0, 2)
After handling missing values, no samples remain.
Abnormality detected in the cohort: GSE64566. Preprocessing failed.
